# YY1-AID TT-seq Differential Expression Analysis

In [1]:
library(DESeq2)
library(tidyverse)
library(RColorBrewer)
library(pheatmap)
library(DEGreport)
library(tximport)
library(ggplot2)
library(ggrepel)

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The follow

## Import data

In [2]:
counts <- 'YY1aid_ttseq_sample_counts.txt'


In [3]:
# read counts generated by deeptools
tab <- read.table(counts, header=TRUE, comment.char='_', strip.white = TRUE,)
names(tab) <- c('gene', 'rep1_untreated', 'rep3_untreated',
               'rep1_auxin','rep3_auxin',
               'rep2_untreated','rep2_auxin')
names(tab)

tab <- data.frame(tab) %>% distinct()


[1] "gene"           "rep1_untreated" "rep3_untreated" "rep1_auxin"    
[5] "rep3_auxin"     "rep2_untreated" "rep2_auxin"

In [4]:
# merge genes with txi
txi <- tab
rownames(txi) <- txi$gene
nrow(txi)

[1] 25208

In [5]:
# make count matrix
txi$counts <- data.matrix(txi[names(txi)[2:7]])


In [6]:
head(txi$counts)

,rep1_untreated,rep3_untreated,rep1_auxin,rep3_auxin,rep2_untreated,rep2_auxin
0610009B22Rik,169,199,198,267,304,447
0610009E02Rik,77,106,85,176,210,288
0610009L18Rik,18,27,8,42,5,20
0610010F05Rik,1565,2160,1812,3221,3445,5366
0610010K14Rik,167,176,181,187,218,267
0610012G03Rik,86,103,100,89,97,106


In [7]:
## Create a sampletable/metadata
sampletype <- factor(c('untreated', 'untreated', 'auxin', 'auxin', 'untreated', 'auxin'))
meta <- data.frame(sampletype, row.names = colnames(txi$counts))
meta$rep <- c('rep1', 'rep3', 'rep1', 'rep3', 'rep2', 'rep2')
meta

,sampletype,rep
,<fct>,<chr>
rep1_untreated,untreated,rep1
rep3_untreated,untreated,rep3
rep1_auxin,auxin,rep1
rep3_auxin,auxin,rep3
rep2_untreated,untreated,rep2
rep2_auxin,auxin,rep2


## Generate normalized counts
note - normalized counts shouldnt be used for DESeq2 step

In [8]:
### Check that sample names match in both files
all(colnames(txi$counts) %in% rownames(meta))
all(colnames(txi$counts) == rownames(meta))

[1] TRUE

[1] TRUE

In [9]:
# create deseq2 object
dds <- DESeqDataSetFromMatrix(countData=txi$counts, colData = meta, design = ~ sampletype+rep)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


In [10]:
# dds <- estimateSizeFactors(dds)

"The sizeFactors vector assigns to each column of the count matrix a value, the size factor, such that count values in the columns can be brought to a common scale by dividing by the corresponding size factor "

# estimate sizefactors w drosophila spike-in

In [11]:
dm6 <- read.table('YY1aid_ttseq_spikein_counts.txt', header=TRUE, comment.char='_', strip.white = TRUE,)
names(dm6) <- c('gene', 'rep1_untreated', 'drop', 'rep3_untreated',
               'rep1_auxin','rep3_auxin',
               'rep2_untreated','rep2_auxin')
dm6 <- dm6[,c('gene', 'rep1_untreated', 'rep3_untreated',
               'rep1_auxin','rep3_auxin',
               'rep2_untreated','rep2_auxin')]

# make count matrix
dm6$counts <- data.matrix(dm6[names(dm6)[2:7]])
rownames(dm6) <- dm6$gene
nrow(dm6)

# create deseq2 object
spike <- DESeqDataSetFromMatrix(countData=dm6$counts, colData = meta, design = ~sampletype)

spike <- estimateSizeFactors(spike)

[1] 17136

In [12]:
data.frame(sizeFactors(spike))

,sizeFactors.spike.
,<dbl>
rep1_untreated,1.4200556
rep3_untreated,0.8806227
rep1_auxin,1.2555608
rep3_auxin,1.0256353
rep2_untreated,0.7027731
rep2_auxin,0.9273714


In [13]:
write.table(data.frame(sizeFactors(spike)), file="./spikein_sizefactors.txt", sep="\t", quote=F, col.names=NA)


In [14]:
sizeFactors(dds) <- sizeFactors(spike)

In [15]:
normalized_counts <- counts(dds, normalized=TRUE)
write.table(normalized_counts, file="./SPIKE_normalized_counts.txt", sep="\t", quote=F, col.names=NA)


## QC

In [16]:
### Transform counts for data visualization
rld <- rlog(dds, blind=TRUE)

In [17]:
### Plot PCA  - SAVE
pdf(file="SPIKE_PCA_normalized_rlogtransform.pdf")

pcaData <- plotPCA(rld, intgroup=c("sampletype", "rep"), returnData=TRUE)
percentVar <- round(100 * attr(pcaData, "percentVar"))
ggplot(pcaData, aes(PC1, PC2, color=sampletype, shape=rep)) +
  geom_point(size=3) +
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance")) + 
  coord_fixed() + 
  theme_classic()

dev.off()

png 
  2

In [18]:
### Extract the rlog matrix from the object
rld_mat <- assay(rld)   

### Compute pairwise correlation values
rld_cor <- cor(rld_mat)    ## cor() is a base R function

In [19]:
rld_cor

,rep1_untreated,rep3_untreated,rep1_auxin,rep3_auxin,rep2_untreated,rep2_auxin
rep1_untreated,1.0000000,0.9996365,0.9996716,0.9993842,0.9993222,0.9990192
rep3_untreated,0.9996365,1.0000000,0.9995150,0.9995331,0.9995253,0.9992316
rep1_auxin,0.9996716,0.9995150,1.0000000,0.9995936,0.9992196,0.9992945
rep3_auxin,0.9993842,0.9995331,0.9995936,1.0000000,0.9993211,0.9995281
rep2_untreated,0.9993222,0.9995253,0.9992196,0.9993211,1.0000000,0.9993642
rep2_auxin,0.9990192,0.9992316,0.9992945,0.9995281,0.9993642,1.0000000


## Run DESeq

In [20]:
dds

class: DESeqDataSet 
dim: 25208 6 
metadata(1): version
assays(1): counts
rownames(25208): 0610009B22Rik 0610009E02Rik ... a ccdc198
rowData names(0):
colnames(6): rep1_untreated rep3_untreated ... rep2_untreated
  rep2_auxin
colData names(3): sampletype rep sizeFactor

In [ ]:
dds <- DESeqDataSetFromMatrix(countData=txi$counts, colData = meta, design = ~ sampletype + rep)
sizeFactors(dds) <- sizeFactors(spike)
dds <- DESeq(dds)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
using pre-existing size factors

estimating dispersions

gene-wise dispersion estimates



In [ ]:
## Plot dispersion estimates
plotDispEsts(dds)

In [ ]:
pdf(file="SPIKE_dispersion_plot.pdf")
plotDispEsts(dds)
dev.off()

## Hypothesis testing

### untreated vs. auxin

In [ ]:
dds$sampletype <- relevel(dds$sampletype, ref = "untreated")
dds <- nbinomWaldTest(dds)

In [ ]:
resultsNames(dds)

In [ ]:
# fold change represented as log2 fold over WT
# alpha set to FDR value, default multiple test correction = bh
# lfcthreshold - p value corresponds to test for 0 log fold change by default
contrast_kd <- c("sampletype","auxin","untreated")

res <- results(dds, contrast=contrast_kd, 
               lfcThreshold = 0,
              alpha = 0.05, pAdjustMethod="BH")

# shrink lfc values for downstream analysis
res <- lfcShrink(dds, res=res, coef=2)

In [ ]:
summary(res, alpha=0.05)

In [ ]:
# pdf(file="MA_plot.pdf")
plotMA(res, ylim=c(-2,2))
# dev.off()



In [ ]:
pdf(file="SPIKE_MA_plot.pdf")
plotMA(res, ylim=c(-2,2))
dev.off()



In [ ]:
### Set thresholds
padj.cutoff <- 0.05

# Create a tibble of results
res_tb <- res %>%
  data.frame() %>%
  rownames_to_column(var="gene") %>% 
  as_tibble()

write.table(res_tb, "SPIKE_ALLgenes_auxin_untreated_p05_TTseq.csv", sep=",", row.names=F)


# Subset the tibble to keep only significant genes
sig <- res_tb %>%
        filter(padj < padj.cutoff)



# Take a quick look at this tibble
# write.table(sig, "significantgenes_auxin_untreated_p05_genebody.csv", sep=",", row.names=F)



## Visualize results

In [ ]:

## Obtain logical vector where TRUE values denote padj values < 0.05 and fold change > 1.5 in either direction
res_tb <- res_tb %>% 
                  mutate(threshold_OE = padj < 0.05 & abs(log2FoldChange) >= log2(1.5))

In [ ]:

## Create an empty column to indicate which genes to label
res_tb <- res_tb %>% mutate(genelabels = "")

## Sort by padj values 
res_tb <- res_tb %>% arrange(padj)

## Populate the genelabels column with contents of the gene symbols column for the first 10 rows, i.e. the top 10 most significantly expressed genes
res_tb$genelabels[1:8] <- as.character(res_tb$gene[1:8])

In [ ]:
ggplot(res_tb, aes(x = log2FoldChange, y = -log10(padj))) +
    geom_point(aes(colour = threshold_OE), size=1) +
    geom_text_repel(aes(label = genelabels)) +
    scale_color_manual(values = c("TRUE" = "firebrick", "FALSE" = "grey")) + 
    ggtitle("TT-seq YY1-AID") +
    xlab("log2 fold change") + 
    ylab("-log10 adjusted p-value") +
    theme_classic() + 
    theme(legend.position = "none",
          plot.title = element_text(size = rel(1.5), hjust = 0.5),
          axis.title = element_text(size = rel(1.25))) 

In [ ]:
pdf("SPIKE_volcano_YY1aid_ttseq.pdf")
ggplot(res_tb, aes(x = log2FoldChange, y = -log10(padj))) +
    geom_point(aes(colour = threshold_OE), size=1) +
    geom_text_repel(aes(label = genelabels)) +
    scale_color_manual(values = c("TRUE" = "firebrick", "FALSE" = "grey")) + 
    ggtitle("TT-seq YY1-AID") +
    xlab("log2 fold change") + 
    ylab("-log10 adjusted p-value") +
    theme_classic() + 
    theme(legend.position = "none",
          plot.title = element_text(size = rel(1.5), hjust = 0.5),
          axis.title = element_text(size = rel(1.25))) 
dev.off()

In [ ]:
sessionInfo()